In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, year, to_date, count, sum as _sum, rank
from pyspark.sql.window import Window
import time

#Αρχικοποίηση SparkSession
spark = SparkSession.builder.appName("Query 2b - CSV/Parquet Comparison").getOrCreate()

#Φόρτωση των δεδομένων από αρχεία CSV
file_path = 's3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv'
file_path2 = 's3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv'
data1 = spark.read.csv(file_path, header=True, inferSchema=True)
data2 = spark.read.csv(file_path2, header=True, inferSchema=True)

#Ένωση των δύο συνόλων δεδομένων
csv_data = data1.union(data2)

#Η μέθοδος coalesce(1) περιορίζει το DataFrame σε ένα μόνο partition
#Αυτό έχει ως αποτέλεσμα να δημιουργηθεί ένα μόνο αρχείο κατά την αποθήκευση των δεδομένων σε αρχείο 
#Parquet, αντί για πολλά αρχεία
single_partition_data = csv_data.coalesce(1)

#Αποθήκευση σε Parquet
parquet_output_path = 's3://groups-bucket-dblab-905418150721/group29/'  #S3 bucket path
single_partition_data.write.mode("overwrite").parquet(parquet_output_path)

print(f"Data successfully saved to {parquet_output_path} in Parquet format.")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Data successfully saved to s3://groups-bucket-dblab-905418150721/group29/ in Parquet format.

In [6]:
### Μετρήσεις χρόνου για CSV ###

#Χρόνος εκτέλεσης για το αρχείο Parquet
start_time_csv = time.time()

#Eξαγωγή του έτους από τη στήλη 'Date Rptd'
csv_data = csv_data.withColumn("year", year(to_date("Date Rptd", "MM/dd/yyyy hh:mm:ss a")))

#Προσθήκη στήλης που ελέγχει αν η υπόθεση έχει κλείσει (δηλαδή δεν είναι "Invest Cont ή UNK")
csv_data = csv_data.withColumn("is_closed",when(~col("Status Desc").rlike("Invest Cont|UNK"), 1).otherwise(0))

#Ομαδοποίηση κατά έτος και όνομα περιοχής για υπολογισμό συνολικών και κλειστών υποθέσεων
csv_summary = csv_data.groupBy("year", "AREA NAME").agg(
    count("DR_NO").alias("total_cases"), #Συνολικές υποθέσεις
    _sum("is_closed").alias("closed_cases") #Κλειστές υποθέσεις
)

#Υπολογισμός ποσοστού κλειστών υποθέσεων
csv_summary = csv_summary.withColumn("closed_case_rate", (col("closed_cases") / col("total_cases")) * 100)

#Ορισμός παραθύρου για κατάταξη με βάση το ποσοστό κλειστών υποθέσεων
window_spec = Window.partitionBy("year").orderBy(col("closed_case_rate").desc())

#Προσθήκη κατάταξης με βάση το ποσοστό κλειστών υποθέσεων
csv_summary = csv_summary.withColumn("rank", rank().over(window_spec))

# Φιλτράρισμα για τις 3 πρώτες εγγραφές ανά έτος
csv_top3_summary = csv_summary.filter(col("rank") <= 3).orderBy("year", "rank")

#Υπολογισμός χρόνου εκτέλεσης
end_time_csv = time.time()
execution_time_csv = end_time_csv - start_time_csv

print(f"CSV Execution Time: {execution_time_csv:.5f} seconds")


#################################################################################################


### Μετρήσεις χρόνου για Parquet ###

#Φόρτωση των δεδομένων από το αρχείο Parquet
parquet_file_path = "s3://groups-bucket-dblab-905418150721/group29/*.snappy.parquet"
parquet_data = spark.read.parquet(parquet_file_path)

#Χρόνος εκτέλεσης για το αρχείο Parquet
start_time_parquet = time.time()

#Eξαγωγή του έτους από τη στήλη 'Date Rptd'
parquet_data = parquet_data.withColumn("year", year(to_date("Date Rptd", "MM/dd/yyyy hh:mm:ss a")))

#Προσθήκη στήλης που ελέγχει αν η υπόθεση έχει κλείσει (δηλαδή δεν είναι "Invest Cont ή UNK")
parquet_data = parquet_data.withColumn("is_closed",when(~col("Status Desc").rlike("Invest Cont|UNK"), 1).otherwise(0))

#Ομαδοποίηση κατά έτος και όνομα περιοχής για υπολογισμό συνολικών και κλειστών υποθέσεων
parquet_summary = parquet_data.groupBy("year", "AREA NAME").agg(
    count("DR_NO").alias("total_cases"),
    _sum("is_closed").alias("closed_cases")
)

#Υπολογισμός ποσοστού κλειστών υποθέσεων
parquet_summary = parquet_summary.withColumn("closed_case_rate", (col("closed_cases") / col("total_cases")) * 100)

#Ορισμός παραθύρου για κατάταξη με βάση το ποσοστό κλειστών υποθέσεων
window_spec = Window.partitionBy("year").orderBy(col("closed_case_rate").desc())

#Προσθήκη κατάταξης με βάση το ποσοστό κλειστών υποθέσεων
parquet_summary = parquet_summary.withColumn("rank", rank().over(window_spec))

# Φιλτράρισμα για τις 3 πρώτες εγγραφές ανά έτος
parquet_top3_summary = parquet_summary.filter(col("rank") <= 3).orderBy("year", "rank")

#Υπολογισμός χρόνου εκτέλεσης
end_time_parquet = time.time()
execution_time_parquet = end_time_parquet - start_time_parquet

print(f"Parquet Execution Time: {execution_time_parquet:.5f} seconds")


#Σύγκριση χρόνων
if execution_time_csv < execution_time_parquet:
    print("CSV was faster.")
else:
    print("Parquet was faster.")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

CSV Execution Time: 0.07734 seconds
Parquet Execution Time: 0.07023 seconds
Parquet was faster.